
# Files

What if the text we want isn't on a webpage? There are a many other sources of
text available, typically organized into *files*.

## Raw text (and encoding)

The most basic form of storing text is as a _raw text_ document. Source code
(`.py`, `.r`, etc) is usually raw text as are text files (`.txt`) and those with
many other extension (e.g., .csv, .dat, etc.). Opening an unknown file with a
text editor is often a great way of learning what the file is.

We can create a text file in python with the `open()` function

Notice the `encoding='utf-8'` argument, which specifies how we map the bits from
the file to the glyphs (and whitespace characters like tab (`'\t'`) or newline
(`'\n'`)) on the screen. When dealing only with latin letters, arabic numerals
and the other symbols on America keyboards you usually do not have to worry
about encodings as the ones used today are backwards compatible with
[ASCII](https://en.wikipedia.org/wiki/ASCII), which gives the binary
representation of 128 characters.

Some of you, however, will want to use other characters (e.g., Chinese
characters). To solve this there is
[Unicode](https://en.wikipedia.org/wiki/Unicode) which assigns numbers to
symbols, e.g., 041 is `'A'` and 03A3 is `'Σ'` (numbers starting with 0 are
hexadecimal). Often non/beyond-ASCII characters are called Unicode characters.
Unicode contains 1,114,112 characters, about 10\% of which have been assigned.
Unfortunately there are many ways used to map combinations of bits to Unicode
symbols. The ones you are likely to encounter are called by Python _utf-8_,
_utf-16_ and _latin-1_. _utf-8_ is the standard for Linux and Mac OS while both
_utf-16_ and _latin-1_ are used by windows. If you use the wrong encoding,
characters can appear wrong, sometimes change in number or Python could raise an
exception. Lets see what happens when we open the file we just created with
different encodings.



In [1]:
#All these packages need to be installed from pip
import requests #for http requests
import bs4 #called `beautifulsoup4`, an html parser
import pandas #gives us DataFrames
import docx #reading MS doc files, install as `python-docx`

#Stuff for pdfs
#Install as `pdfminer2`
import pdfminer.pdfinterp
import pdfminer.converter
import pdfminer.layout
import pdfminer.pdfpage

#These come with Python
import re #for regexs
import urllib.parse #For joining urls
import io #for making http requests look like files
import json #For Tumblr API responses
import os.path #For checking if files exist
import os #For making directories

In [2]:
information_extraction_pdf = 'https://github.com/Computational-Content-Analysis-2018/Data-Files/raw/master/1-intro/Content%20Analysis%2018.pdf'
example_docx = 'https://github.com/Computational-Content-Analysis-2018/Data-Files/raw/master/1-intro/macs6000_connecting_to_midway.docx'
example_docx_save = 'example.docx'

In [3]:
example_text_file = 'sometextfile.txt'
#stringToWrite = 'A line\nAnother line\nA line with a few unusual symbols \u2421 \u241B \u20A0 \u20A1 \u20A2 \u20A3 \u0D60\n'
stringToWrite = 'A line\nAnother line\nA line with a few unusual symbols ␡ ␛ ₠ ₡ ₢ ₣ ൠ\n'

with open(example_text_file, mode = 'w', encoding='utf-8') as f:
    f.write(stringToWrite)

In [4]:
with open(example_text_file, encoding='utf-8') as f:
    print("This is with the correct encoding:")
    print(f.read())

with open(example_text_file, encoding='latin-1') as f:
    print("This is with the wrong encoding:")
    print(f.read())

This is with the correct encoding:
A line
Another line
A line with a few unusual symbols ␡ ␛ ₠ ₡ ₢ ₣ ൠ

This is with the wrong encoding:
A line
Another line
A line with a few unusual symbols â¡ â â  â¡ â¢ â£ àµ 



We can also load many text files at once. Lets tart by looking at the Shakespeare files in the `data` directory 

In [5]:
with open('../../data/Shakespeare/midsummer_nights_dream.txt') as f:
    midsummer = f.read()
print(midsummer[-700:])

, and Train.]

PUCK
  If we shadows have offended,
  Think but this,--and all is mended,--
  That you have but slumber'd here
  While these visions did appear.
  And this weak and idle theme,
  No more yielding but a dream,
  Gentles, do not reprehend;
  If you pardon, we will mend.
  And, as I am an honest Puck,
  If we have unearned luck
  Now to 'scape the serpent's tongue,
  We will make amends ere long;
  Else the Puck a liar call:
  So, good night unto you all.
  Give me your hands, if we be friends,
  And Robin shall restore amends.

[Exit.]





End of Project Gutenberg Etext of A Midsummer Night's Dream by Shakespeare
PG has multiple editions of William Shakespeare's Complete Works



Then to load all the files in `../data/Shakespeare` we can use a for loop with `scandir`:

In [6]:
targetDir = '../../data/Shakespeare' #Change this to your own directory of texts
shakespearText = []
shakespearFileName = []

for file in (file for file in os.scandir(targetDir) if file.is_file() and not file.name.startswith('.')):
    with open(file.path, encoding="utf-8") as f:
        shakespearText.append(f.read())
    shakespearFileName.append(file.name)

Then we can put them all in pandas DataFrame

In [7]:
shakespear_df = pandas.DataFrame({'text' : shakespearText}, index = shakespearFileName)
shakespear_df

,text
julius_caesar.txt,"Dramatis Personae\n\n JULIUS CAESAR, Roman st..."
as_you_like_it.txt,AS YOU LIKE IT\n\nby William Shakespeare\n\n\n...
tempest.txt,"The Tempest\n\nActus primus, Scena prima.\n\nA..."
phoenix_and_the_turtle.txt,THE PHOENIX AND THE TURTLE\n\nby William Shake...
king_lear.txt,The Tragedie of King Lear\n\n\nActus Primus. S...
passionate_pilgrim.txt,THE PASSIONATE PILGRIM\n\nby William Shakespea...
cymbeline.txt,The Tragedie of Cymbeline\n\nActus Primus. Sco...
coriolanus.txt,THE TRAGEDY OF CORIOLANUS\n\nby William Shakes...
two_gentlemen_of_verona.txt,THE TWO GENTLEMEN OF VERONA\n\nby William Shak...
rape_of_lucrece.txt,THE RAPE OF LUCRECE\n\nby William Shakespeare\...


Getting your text in a format like this is the first step of most analysis

## PDF

Another common way text will be stored is in a PDF file. First we will download
a pdf in Python. To do that lets grab a chapter from
_Speech and Language Processing_, chapter 21 is on Information Extraction which
seems apt. It is stored as a pdf at [https://web.stanford.edu/~jurafsky/slp3/21.
pdf](https://web.stanford.edu/~jurafsky/slp3/21.pdf) although we are downloading
from a copy just in case Jurafsky changes their website.

In [8]:
#information_extraction_pdf = 'https://github.com/KnowledgeLab/content_analysis/raw/data/21.pdf'

infoExtractionRequest = requests.get(information_extraction_pdf, stream=True)
print(infoExtractionRequest.text[:1000])

%PDF-1.3
%���������
4 0 obj
<< /Length 5 0 R /Filter /FlateDecode >>
stream
x�]۶�F�}�W�c����T���C�i�<t�bM�f
Tn�<3_��CDf��J�N�i��#�%.;.���	?߄��7�]8������ux��}޾m����y��bǾ����!���$�Ǯ���C��F�����p���5��1��1�P<�{�$��/$�P�s�v��PgH?�����Q�~�*�:l��ˇ�m�ǰ��C�l����܊��E�����!�^�y��m�$�Ý���wۡل׼�6w���ī�K�~؞���r��~?�ˡkO�;6IH�9{ԡ��� ]?�E�E��~���.l������+��W�_�����C��S�|�~C��N�3ӛB`8�ޚj9���AZ��0�d�l^������SY	�Ƨ��>q�ۇ&
����.�����0����; ��>a8�$w�p��p����ST���. �7��@����)��&1�|���WՃ jOv�G2b�L8I��N�@gǍ������O�C��������IN@@�����}�8��+L����a�&ү�o�V(���0���+5�
SfS&��<�2�����>l�V��&��=4⇤=�W��<�JMo����"����d�C����[vY�|K{_ܔ\����%H�/@'�QA�+D�l��c��L�G�.��	�̎�V�:f>���AwK���o$`D��bE45�0�%th6h�����>*�2vQd�+M��Y}�Q���u�[���N�o'b��/u�.r'Z���J�e8�v��;��{T�	�����^8�  l<�E�<���b�����C8j��f��xB>K�����|w��f�|?�s̭��Y�'�Ip&�"�A���f�?�!IYi���U�"��y;���#��e3)�+B�&����<E9I�g�/]"D��yfC;e����Y^�z ��s'�)/�X�-HY��<ˬ�ݰ

It says `'pdf'`, so thats a good sign. The rest though looks like we are having
issues with an encoding. The random characters are not caused by our encoding
being wrong, however. They are cause by there not being an encoding for those
parts at all. PDFs are nominally binary files, meaning there are sections of
binary that are specific to pdf and nothing else so you need something that
knows about pdf to read them. To do that we will be using
[`PyPDF2`](https://github.com/mstamy2/PyPDF2), a PDF processing library for
Python 3.


Because PDFs are a very complicated file format pdfminer requires a large amount
of boilerplate code to extract text, we have written a function that takes in an
open PDF file and returns the text so you don't have to.

In [12]:
def readPDF(pdfFile):
    #Based on code from http://stackoverflow.com/a/20905381/4955164
    #Using utf-8, if there are a bunch of random symbols try changing this
    codec = 'utf-8'
    rsrcmgr = pdfminer.pdfinterp.PDFResourceManager()
    retstr = io.StringIO()
    layoutParams = pdfminer.layout.LAParams()
    device = pdfminer.converter.TextConverter(rsrcmgr, retstr, laparams = layoutParams)
    #We need a device and an interpreter
    interpreter = pdfminer.pdfinterp.PDFPageInterpreter(rsrcmgr, device)
    password = ''
    maxpages = 0
    caching = True
    pagenos=set()
    for page in pdfminer.pdfpage.PDFPage.get_pages(pdfFile, pagenos, maxpages=maxpages, password=password,caching=caching, check_extractable=True):
        interpreter.process_page(page)
    device.close()
    returnedString = retstr.getvalue()
    retstr.close()
    return returnedString

First we need to take the response object and convert it into a 'file like'
object so that pdfminer can read it. To do this we will use `io`'s `BytesIO`.

In [10]:
infoExtractionBytes = io.BytesIO(infoExtractionRequest.content)

Now we can give it to pdfminer.

In [13]:
print(readPDF(infoExtractionBytes)[:550])

Department of  Sociology 

THE UNIVERSITY OF CHICAGO 

SOCIOLOGY 40133 

Computational Content Analysis 

Friday 1:00 – 3:50pm 
Winter 2017-2018 
Classroom: Harper Memorial 130       
http://chalk.uchicago.edu/ 

 

                                                                                           

          Office: McGiffert 210 
                                                    Tel.: 834-3612; jevans@uchicago.edu 
                                  Office Hours: Thursday 12:30-2:30pm 

     

        James A. Evans            

    


From here we can either look at the full text or fiddle with our PDF reader and
get more information about individual blocks of text.

## Word Docs

The other type of document you are likely to encounter is the `.docx`, these are
actually a version of [XML](https://en.wikipedia.org/wiki/Office_Open_XML), just
like HTML, and like HTML we will use a specialized parser.

For this class we will use [`python-docx`](https://python-
docx.readthedocs.io/en/latest/) which provides a nice simple interface for
reading `.docx` files

In [14]:
#example_docx = 'https://github.com/KnowledgeLab/content_analysis/raw/data/example_doc.docx'

r = requests.get(example_docx, stream=True)
d = docx.Document(io.BytesIO(r.content))
for paragraph in d.paragraphs[:7]:
    print(paragraph.text)

 
 

Accessing the Research Computing Center Resources

To connect to the midway compute cluster to access your home directory and the macs60000 storage space, and utilize the HPC resources, you will either use a terminal client (with or without X11 forwarding capabilities) or the Linux remote desktop server software client (Thinlinc) to connect to the midway cluster. To submit jobs, monitor jobs, browse directories or do other computing you will need to connect through either the terminal or remote desktop. Setup and utilization of these clients will be discussed below in the context of your local platform’s architecture.
SSH Client Setup & Remote Desktop Server


This procedure uses the `io.BytesIO` class again, since `docx.Document` expects
a file. Another way to do it is to save the document to a file and then read it
like any other file. If we do this we can either delete the file afterwords, or
save it and avoid downloading the following time.

This function is useful as a part of many different tasks so it and others like it will be added to the helper package `lucem_illud_2020` so we can use it later without having to retype it.

In [15]:
def downloadIfNeeded(targetURL, outputFile, **openkwargs):
    if not os.path.isfile(outputFile):
        outputDir = os.path.dirname(outputFile)
        #This function is a more general os.mkdir()
        if len(outputDir) > 0:
            os.makedirs(outputDir, exist_ok = True)
        r = requests.get(targetURL, stream=True)
        #Using a closure like this is generally better than having to
        #remember to close the file. There are ways to make this function
        #work as a closure too
        with open(outputFile, 'wb') as f:
            f.write(r.content)
    return open(outputFile, **openkwargs)

This function will download, save and open `outputFile` as `outputFile` or just
open it if `outputFile` exists. By default `open()` will open the file as read
only text with the local encoding, which may cause issues if its not a text
file.

In [16]:
try:
    d = docx.Document(downloadIfNeeded(example_docx, example_docx_save))
except Exception as e:
    print(e)

File is not a zip file


We need to tell `open()` to read in binary mode (`'rb'`), this is why we added
`**openkwargs`, this allows us to pass any keyword arguments (kwargs) from
`downloadIfNeeded` to `open()`.

In [17]:
d = docx.Document(downloadIfNeeded(example_docx, example_docx_save, mode = 'rb'))
for paragraph in d.paragraphs[:7]:
    print(paragraph.text)

 
 

Accessing the Research Computing Center Resources

To connect to the midway compute cluster to access your home directory and the macs60000 storage space, and utilize the HPC resources, you will either use a terminal client (with or without X11 forwarding capabilities) or the Linux remote desktop server software client (Thinlinc) to connect to the midway cluster. To submit jobs, monitor jobs, browse directories or do other computing you will need to connect through either the terminal or remote desktop. Setup and utilization of these clients will be discussed below in the context of your local platform’s architecture.
SSH Client Setup & Remote Desktop Server


Now we can read the file with `docx.Document` and not have to wait for it to be
downloaded every time.